# Part 04: Tabulate statistics on the street disconntinuties
michael babb  
2024 11 24

In [2]:
# standard
import os

In [3]:
# external
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import shapely.geometry
from shapely.geometry import LineString, MultiLineString

In [4]:
# custom
import run_constants as rc
from utils import *

# output

In [5]:
ofpn = os.path.join(rc.ANALYSIS_OUTPUT_FILE_PATH, rc.S04_ANALYSIS_OUT_FILE_NAME)

In [6]:
e_writer = pd.ExcelWriter(path = ofpn)

# load the street network data

In [7]:
fpn = os.path.join(rc.OUTPUT_FILE_PATH, rc.S04_MISSING_IN_FILE_NAME)

In [8]:
gdf = gpd.read_file(filename = fpn)

In [9]:
gdf.columns

Index(['snd_id', 'ord_street_name', 'ord_stname_concat', 'ord_street_type',
       'ord_street_type_rank', 'snd_group', 'street_status', 'sn_id', 'en_id',
       'dist', 'dist_miles', 'geometry'],
      dtype='object')

In [10]:
gdf['n_segments'] = int(1)

In [11]:
# add text descriptions for the street type categories
missing_dict = {0:'complete_street',
                1:'disconnected_street',
                2:'missing_street'}

In [12]:
gdf['street_status_desc'] = gdf['street_status'].map(missing_dict)

# export data for use in a web map

In [33]:
# create a function to write json data with a var name

def write_json(json_data:str, output_file_path:str,
               output_file_name:str):
    
    ofpn = os.path.join(output_file_path, output_file_name)
    var_name_str = os.path.splitext(output_file_name)[0]
    print(var_name_str)
    with open(ofpn, 'w') as file:    
        write_line = 'var {} ='.format(var_name_str)
        file.write(write_line)
        file.write(json_data)
    print('here')
    return None

In [34]:
gdf_list = []
for ss in gdf['street_status_desc'].unique().tolist():
    print(ss)
    output_file_name = ss + 's.geojson'
    
    tgdf = gdf.loc[gdf['street_status_desc'] == ss, :].copy()
    tgdf = tgdf.iloc[:10].copy()
        
    out_data = tgdf.to_json(drop_id=True, to_wgs84=True)

    write_json(json_data=out_data, output_file_path ='../maps',
               output_file_name = output_file_name)
    
    gdf_list.append(tgdf)
    

missing_street
missing_streets
here
disconnected_street
disconnected_streets
here
complete_street
complete_streets
here


In [35]:
# write a subset of the test streets
tgdf = pd.concat(gdf_list)
out_data = tgdf.to_json(drop_id=True, to_wgs84=True)
output_file_name = 'all_streets_test.geojson'
write_json(json_data=out_data, output_file_path ='../maps',
               output_file_name = output_file_name)

all_streets_test
here


In [36]:
# write out all of the streets - but this has all of the data - we don't need it
out_data = gdf.to_json(drop_id=True, to_wgs84=True)
output_file_name = 'all_streets.geojson'
write_json(json_data=out_data, output_file_path ='../maps',
               output_file_name = output_file_name)
    

all_streets
here


In [37]:
col_names = ['ord_stname_concat', 'snd_group', 'street_status', 'dist_miles', 'geometry']
diss_gdf = gdf[col_names].dissolve(by = col_names[:-2:],
                     aggfunc = 'sum', as_index = False)

In [38]:
diss_gdf.head()

,ord_stname_concat,snd_group,street_status,geometry,dist_miles
0,10TH AVE,0,2,"LINESTRING (-122.31942 47.60311, -122.31936 47...",0.025645
1,10TH AVE,1,2,"LINESTRING (-122.31943 47.60621, -122.31938 47...",0.209270
2,10TH AVE,2,2,"LINESTRING (-122.31937 47.61041, -122.31936 47...",0.093511
3,10TH AVE,3583,1,"MULTILINESTRING ((-122.31942 47.6027, -122.319...",0.101482
4,10TH AVE,3584,1,"MULTILINESTRING ((-122.31942 47.60352, -122.31...",0.190358


In [39]:
# does every MultiLineString need to be a MultilineString?
def check_MultiLineStrings(ls:shapely.geometry):

In [40]:
out_data = diss_gdf.to_json(drop_id=True, to_wgs84=True)
output_file_name = 'all_streets_diss.geojson'
write_json(json_data=out_data, output_file_path ='../maps',
               output_file_name = output_file_name)


all_streets_diss
here


# what street name - excluding direction - has the most street types?

In [135]:
col_names = ['ord_street_name', 'ord_street_type', 'n_segments']

In [136]:
gdf_agg = gdf[col_names].drop_duplicates()

In [137]:
gdf_pv = pd.pivot_table(data = gdf_agg, values = ['n_segments'], index = 'ord_street_name', columns = ['ord_street_type'], aggfunc = 'sum', fill_value = 0, margins = True).reset_index()

In [138]:
gdf_pv.columns = [''.join(cn).replace('n_segments', '') for cn in gdf_pv.columns]

In [ ]:
gdf_pv.head()

In [140]:
gdf_pv.to_excel(excel_writer=e_writer, sheet_name = 's_type_count', index = False)

# what street name - excluding type - has the most street connections?

In [141]:
col_names = ['ord_street_name', 'ord_stname_concat', 'street_status_desc', 'n_segments']
gdf_agg = gdf[col_names].drop_duplicates()

In [142]:
gdf_pv = pd.pivot_table(data = gdf_agg, values = ['n_segments'], index = 'ord_street_name', columns = ['street_status_desc'],
                        aggfunc = 'sum', fill_value = 0, margins = True).reset_index()

In [ ]:
gdf_pv.head()

In [144]:
gdf_pv.columns = [''.join(cn).replace('n_segments', '') for cn in gdf_pv.columns]

In [ ]:
gdf_pv.columns

In [ ]:
gdf_pv.head()

In [147]:
gdf_pv.to_excel(excel_writer=e_writer, sheet_name = 'connection_count_by_name', index = False)

# what street name - including direction and type - has the most street connections?

In [148]:
col_names = ['ord_stname_concat', 'ord_street_type', 'street_status', 'street_status_desc', 'n_segments']
gdf_agg = gdf[col_names]

In [149]:
gdf_pv = pd.pivot_table(data = gdf_agg, values = ['n_segments'], index = ['ord_stname_concat', 'ord_street_type'],
                        columns = ['street_status_desc'], aggfunc = 'sum', fill_value = 0, margins = True).reset_index()

In [150]:
gdf_pv.columns = [''.join(cn).replace('n_segments', '') for cn in gdf_pv.columns]

In [ ]:
gdf_pv.head()

In [152]:
# remove the all row
gdf_pv = gdf_pv.loc[gdf_pv['ord_stname_concat'] != 'All', :]

In [ ]:
gdf_pv.shape

# what are the streets with at least 10 discontinuities?

In [ ]:
gdf_pv.loc[gdf_pv['missing_street'] >= 10, 'ord_stname_concat'].tolist()

In [155]:
gdf_pv.to_excel(excel_writer=e_writer, sheet_name = 'connection_count_full_name', index = False)

# what is ratio of disconnected to complete streets?

In [ ]:
# number of streets with at least one missing segment
split_streets = gdf_pv.loc[gdf_pv['disconnected_street'] > 0, :].shape[0]
split_streets

In [ ]:
# number of streets with no missing segments
complete_streets = gdf_pv.loc[gdf_pv['complete_street'] > 0, :].shape[0]
complete_streets

In [ ]:
split_streets / gdf_pv.shape[0]

In [ ]:
complete_streets / gdf_pv.shape[0]

In [160]:
# 46 percent of streets are multi-segments.
# 54 percent of streets are single segment.

# sum of road miles by disconnected street

In [161]:
col_names = ['ord_stname_concat', 'ord_street_type', 'street_status', 'street_status_desc', 'dist_miles']
gdf_agg = gdf[col_names]

In [162]:
gdf_pv = pd.pivot_table(data = gdf_agg, values = ['dist_miles'], index = ['ord_stname_concat', 'ord_street_type'],
                        columns = ['street_status_desc'], aggfunc = 'sum', fill_value = 0, margins = True).reset_index()

In [163]:
gdf_pv.columns = [''.join(cn).replace('dist_miles', '') for cn in gdf_pv.columns]

In [ ]:
gdf_pv.shape

In [165]:
gdf_pv = gdf_pv.loc[gdf_pv['ord_stname_concat']!= 'All', :].copy()

In [ ]:
gdf_pv.shape

In [ ]:
gdf_pv.head()

# which streets feature longer disconnected portions?

In [168]:
gdf_pv['longer_missing'] = int(0) # street is complete
gdf_pv.loc[(gdf_pv['complete_street'] == 0) & (gdf_pv['missing_street'] <= gdf_pv['disconnected_street']), 'longer_missing'] = int(1)
gdf_pv.loc[(gdf_pv['complete_street'] == 0) & (gdf_pv['missing_street'] > gdf_pv['disconnected_street']), 'longer_missing'] = int(2)

In [ ]:
gdf_pv['longer_missing'].value_counts()

In [170]:
gdf_pv.to_excel(excel_writer=e_writer, sheet_name = 'distance_full_name', index = False)

In [ ]:
# road miles on streets with at least one missing segment
split_streets = gdf_pv['disconnected_street'].sum()
split_streets

In [ ]:
# road miles on streets with no missing segments
complete_streets = gdf_pv['complete_street'].sum()
complete_streets

In [ ]:
split_streets / (split_streets + complete_streets)

In [ ]:
complete_streets / (split_streets + complete_streets)

In [175]:
# 78 percent of road miles are multi-segments streets
# 22 percent of road miles are single segment streets

# tabulate distances and distributions by street type

In [176]:
st_type_df = pd.pivot_table(data = gdf, index = ['ord_street_type'],
                         columns = ['street_status_desc'], values = ['dist_miles'],
                        aggfunc='sum', fill_value = 0).reset_index()



In [177]:
st_type_df.columns = [''.join(cn).replace('dist_miles', '') for cn in st_type_df.columns]

In [ ]:
st_type_df.shape

In [ ]:
st_type_df.head()

In [180]:
# now, melt
st_type_df_melt = pd.melt(frame = st_type_df, id_vars = ['ord_street_type'],
                          var_name = 'street_status_desc',
                          value_name = 'total_miles')

In [ ]:
st_type_df_melt.head()

In [182]:
col_names = ['ord_street_type', 'street_status_desc', 'dist_miles']

In [183]:
ds_st_df = gdf[col_names].groupby(col_names[:-1]).describe().reset_index()

In [ ]:
ds_st_df.head()

In [185]:
ds_st_df.columns =  [''.join(cn).replace('dist_miles', '') for cn in ds_st_df.columns]

In [ ]:
ds_st_df.head()

In [187]:
temp_ds_st_df = ds_st_df.loc[ds_st_df['street_status_desc'] == 'missing_street', :].copy()

In [188]:
temp_ds_st_df = temp_ds_st_df.drop(labels = 'std', axis = 1)

In [189]:
temp_ds_st_df = temp_ds_st_df.sort_values(by = ['count'], ascending = False)

In [190]:
temp_ds_st_df.to_excel(excel_writer=e_writer, sheet_name = 'missing_street_count', index = False)

In [ ]:
st_type_df_melt.head()

In [ ]:
ds_st_df.head()

In [193]:
# join to get total miles and summary statistics
ds_st_df = pd.merge(left = st_type_df_melt, right = ds_st_df, how = 'left')

In [ ]:
ds_st_df.head()

In [ ]:
ds_st_df.columns

In [196]:
for cn in ds_st_df.columns:
    if ds_st_df[cn].dtype == 'float64':
        ds_st_df[cn] = ds_st_df[cn].fillna(0)

In [197]:
ds_st_df = ds_st_df.sort_values(by = ['ord_street_type', 'street_status_desc'])

In [198]:
ds_st_df['street_status_desc'] = ds_st_df['street_status_desc'].str.replace('_street', '')

In [199]:
# set index

In [200]:
ds_st_df = ds_st_df.set_index(keys = ['ord_street_type', 'street_status_desc'])

In [201]:
# reorder columns
col_names = ['count', 'min', '25%', '50%', 'mean', '75%', 'max', 'total_miles']
ds_st_df = ds_st_df[col_names]

In [202]:
ds_st_df.to_excel(excel_writer=e_writer, sheet_name = 'sum_stats', index = True)
                  

In [203]:
e_writer.close()

In [ ]:
ds_st_df.head()

# 10 longest missing streets

In [205]:
wms_gdf = gdf.loc[gdf['street_status'] == 2, :].copy()

In [206]:
wms_gdf['dist_rank'] = wms_gdf['dist'].rank(method = 'dense', ascending = False)

In [ ]:
wms_gdf.loc[wms_gdf['dist_rank'] <= 10, 'ord_stname_concat'].tolist()

# histogram of all added streets

In [208]:
# log transform the distance in miles to prep for plotting
wms_gdf['dist_miles_log'] = np.log10(wms_gdf['dist_miles'])

In [ ]:
wms_gdf['dist_miles_log'].describe()

In [210]:
# compute summary stats and create a dataframe
# the ds_df will hold values of interest to plot and plotting keywords for 
# those values
ds_df = wms_gdf['dist_miles'].describe().to_frame().reset_index(names = ['stat'])

In [211]:
# remove the count and the standard deviation - not useful for plotting
ds_df = ds_df.loc[-ds_df['stat'].isin(['count', 'std']), :]

In [ ]:
# add the 10th, 95th, and 99th precentiles
temp_stat_records = [
    ['10%', np.quantile(a = wms_gdf['dist_miles'], q = 0.10)],
    ['95%', np.quantile(a = wms_gdf['dist_miles'], q = 0.95)],
    ['99%', np.quantile(a = wms_gdf['dist_miles'], q = 0.99)]
]
temp_stat_df = pd.DataFrame(data = temp_stat_records, columns = ds_df.columns)
temp_stat_df.head()

In [213]:
# combine
ds_df = pd.concat(objs = [ds_df, temp_stat_df], axis = 0)

In [214]:
ds_df = ds_df.sort_values(by = ['dist_miles'])

In [215]:
ds_df['dist_miles_log'] = ds_df['dist_miles'].map(lambda x: np.log10(x))

In [216]:
# the color of the lines to add
ds_df['color_list'] = 'black'

In [217]:
# convert to feet to help with labeling
ds_df['dist_feet'] = ds_df['dist_miles'] * 5280

In [218]:
# this the horizontal alignment of each piece of added text
ds_df['text_ha'] = ['left', 'right',  'center', 'right', 'right', 'left', 'center', 'center', 'right']

In [219]:
# the text labels to add
ds_df['text'] = ['4ft (min.)', '99 ft (10%)', '172 ft (25%)', '443 ft (med.)', '0.23 mi (avg.)', '0.25 mi (75%)', '0.96 mi (95%)','1.8 mi (99%)', '5 mi (max.)']

In [220]:
# jitter each piece of text by a small amount left or right
ds_df['text_jitter'] = [0, .001, 0, .09, 0, 0, -.05, 0, 0]

In [221]:
# the horizontal position is created from the distance in miles and the text jitter
ds_df['text_x_pos'] = ds_df['dist_miles_log'] + ds_df['text_jitter']

In [ ]:
ds_df.head(n=10)

In [ ]:
wms_gdf.head()

In [ ]:
# compute the histogram of the added distribution
sns.set_theme(style = "whitegrid")
f, ax = plt.subplots(figsize = (20, 5))

ax.set_ylim(0, 185.6)
ax.set_xlim(-3.2, .75)

my_plot = sns.histplot(data = wms_gdf, x = 'dist_miles_log',
                      color='#ca0020', bins = 100)

# set the top part of the plot frame - the spine - to black
ax.spines['top'].set_edgecolor(color = 'black')

y_ticks = list(range(0, 176, 25))
y_tick_labels_formatted = ['{:,}'.format(ytl) for ytl in y_ticks]   

my_plot.set_yticks(ticks = y_ticks)
my_plot.set_yticklabels(labels = y_tick_labels_formatted, rotation=0)

# total missing streets
n_missing = wms_gdf.shape[0]
n_missing =  f"{n_missing:,}"

tot_miles = wms_gdf['dist_miles'].sum()
tot_miles = f"{int(round(tot_miles, 0)):,}"

my_title = "Histogram of missing road segment length, all road types (n = {}, {} miles)".format(n_missing, tot_miles)

plt.title(label = my_title, fontsize = 16)
plt.xlabel(xlabel = 'Missing road segment length (log-scale)')
plt.ylabel(ylabel = "Number of misssing road segments")

# x tick positions
x_tick_labels = [50 / 5280, 100 / 5280, 300 / 5280, .1, .25, .5, 1, 2, 3]
x_ticks = [np.log10(xtl) for xtl in x_tick_labels]

# x tick labels
x_tick_label_text = ['50 ft', '100 ft', '300 ft',  '1/10 mi', '1/4 mi', '1/2 mi', '1 mi', '2 mi', '3 mi']


my_plot.set_xticks(ticks = x_ticks)
my_plot.set_xticklabels(labels = x_tick_label_text, rotation=0)

# white out the top portion of the ticks
for xt in x_ticks:
    my_plot.vlines(x = xt, ymin = 175, ymax = 185, color = 'white', linestyles = 'solid', linewidth = 2)

# add vertical lines at some descriptive statistics points
my_plot.vlines(x = ds_df['dist_miles_log'], ymin = 0, ymax = 175, color = ds_df['color_list'], linestyles = 'dashdot')

# add text for the descriptive stats
for ir, row in ds_df.iterrows():
    my_plot.text(x = row['text_x_pos'], y = 176, s = row['text'], horizontalalignment = row['text_ha'], color = '#ca0020')

output_file_name = '..\\graphics\\histogram_ALL_streets.png'
output_file_name = os.path.normpath(output_file_name)
print(output_file_name)
my_plot.get_figure().savefig(fname = output_file_name)
plt.show()

In [225]:
# and there you have it... most added streets are short. 
# 75% of all added streets are 0.25 miles or less. 

# histogram of each type of street

In [226]:
st_type_df = pd.pivot_table(data = wms_gdf, index = ['ord_street_type'],
                         columns = ['street_status_desc'], values = ['dist_miles'],
                        aggfunc='sum', fill_value = 0).reset_index()

In [ ]:
st_type_df.head()

In [228]:
st_type_df.columns = [''.join(cn).replace('dist_miles', '') for cn in st_type_df.columns]

In [ ]:
st_type_df.head()

In [230]:
st_type_df.columns = ['ord_street_type', 'total_miles']

In [231]:
ds_df = wms_gdf[['ord_street_type', 'dist_miles']].groupby(['ord_street_type'])['dist_miles'].describe().reset_index(names = ['ord_street_type'])

In [ ]:
ds_df.head()

In [ ]:
ds_df['max'].tolist()

In [ ]:
ds_df.shape

In [ ]:
for ir, row in ds_df.iterrows():
    # let's make a graphic showing the distances
    sns.set_theme(style = "whitegrid")
    f, ax = plt.subplots(figsize = (20, 5))

    # set the top part of the plot frame - the spine - to black
    ax.spines['top'].set_edgecolor(color = 'black')

    x_max = row['max']
    ost = row['ord_street_type']    
    
    if x_max > 1:
        xlim_max = .75
    else:
        xlim_max = 0
    ax.set_xlim(-3.2, xlim_max)

    temp_gdf = wms_gdf.loc[wms_gdf['ord_street_type'] == ost, :]
    my_plot = sns.histplot(data = temp_gdf, x = 'dist_miles_log',
                          color='#ca0020', bins = 100)
    
    # total missing streets
    n_missing = temp_gdf.shape[0]
    n_missing =  f"{n_missing:,}"

    tot_miles = temp_gdf['dist_miles'].sum()    
    dist_units = 'miles'
    if tot_miles < .25:
        tot_dist = int(round(temp_gdf['dist'].sum(), 0))
        dist_units = 'feet'        
    elif tot_miles >= .25 and tot_miles <= 5:    
        tot_dist = tot_miles
        tot_dist = round(tot_dist, 2)        
    else:
        tot_dist = int(round(tot_miles, 0))        
        
    my_title =   "Histogram of missing road segment length: {} (n = {}, {} {})".format(ost, n_missing, tot_dist, dist_units )
    
    plt.title(label = my_title, fontsize = 16)
    plt.xlabel(xlabel = 'Missing road segment  length (log-scale)')
    plt.ylabel(ylabel = "Number of misssing road segments")
    
    if x_max > 1:
        x_tick_labels = [50 / 5280, 100 / 5280, 300 / 5280, .1, .25, .5, 1, 2, 3]
        x_tick_label_text = ['50 ft', '100 ft', '300 ft',  '1/10 mi', '1/4 mi', '1/2 mi', '1 mi', '2 mi', '3 mi']
    else:
        x_tick_labels = [50 / 5280, 100 / 5280, 300 / 5280, .1, .25, .5, 1]
        x_tick_label_text = ['50 ft', '100 ft', '300 ft',  '1/10 mi', '1/4 mi', '1/2 mi', '1 mi']
        
    x_ticks = [np.log10(xtl) for xtl in x_tick_labels]
    
    my_plot.set_xticks(ticks = x_ticks)
    my_plot.set_xticklabels(labels = x_tick_label_text, rotation=0)    
    
    
    output_file_name = '..\\graphics\\histogram_{}.png'.format(row['ord_street_type'])
    output_file_name = os.path.normpath(output_file_name)
    print(output_file_name)
    my_plot.get_figure().savefig(fname = output_file_name)
    plt.close()

In [ ]:
ds_st_df.head()

In [237]:
col_names = ['ord_street_type', 'street_status_desc', 'dist_miles']
ds_st_df = gdf[col_names].groupby(col_names[:-1]).agg(n_segments = ('dist_miles', 'size'),
                                                      n_miles = ('dist_miles', 'sum')).reset_index()

In [ ]:
ds_st_df.head()

In [239]:
col_names = ['street_status_desc', 'dist_miles']
all_ds_st_df = gdf[col_names].groupby(col_names[:-1]).agg(n_segments = ('dist_miles', 'size'),
                                                      n_miles = ('dist_miles', 'sum')).reset_index()

In [240]:
all_ds_st_df['ord_street_type'] = 'ALL'

In [241]:
col_names = ['ord_street_type', 'street_status_desc', 'n_segments', 'n_miles']
all_ds_st_df = all_ds_st_df[col_names]

In [242]:
ds_st_df = pd.concat(objs = [all_ds_st_df, ds_st_df])

In [ ]:
ds_st_df.head()

In [244]:
ds_st_df['n_miles_log'] = np.log(ds_st_df['n_miles'])
ds_st_df['street_status_desc'] = ds_st_df['street_status_desc'].str.replace('_', ' ')

In [ ]:
sns.set_theme(style = "whitegrid")
f, ax = plt.subplots(figsize = (20, 5))
ax.set_ylim(0, 1600)

# these are the same green, black, and red colors used in qGIS. 
my_color_palette = {'complete street': '#33a02c',
'disconnected street': '#000000',
'missing street': '#ca0020'}

my_plot = sns.barplot(data = ds_st_df, x = 'ord_street_type', y = 'n_miles',
                      hue = 'street_status_desc', palette=my_color_palette)

# set the top part of the plot frame - the spine - to black
ax.spines['top'].set_edgecolor(color = 'black')

y_ticks = list(range(0, 1601, 200))
y_tick_labels_formatted = ['{:,}'.format(ytl) for ytl in y_ticks]   

my_plot.set_yticks(ticks = y_ticks)
my_plot.set_yticklabels(labels = y_tick_labels_formatted, rotation=0)

my_title = "Total miles by road types and road status"

plt.title(label = my_title, fontsize = 16)
plt.xlabel(xlabel = 'Road type')
plt.ylabel(ylabel = "Miles")

plt.legend(title='Road status')

output_file_name = '..\\graphics\\barplot_miles.png'
output_file_name = os.path.normpath(output_file_name)
print(output_file_name)
my_plot.get_figure().savefig(fname = output_file_name)
plt.show()

In [ ]:
sns.set_theme(style = "whitegrid")
f, ax = plt.subplots(figsize = (20, 5))
ax.set_ylim(0, 22000)

# these are the same green, black, and red colors used in qGIS. 
my_color_palette = {'complete street': '#33a02c',
'disconnected street': '#000000',
'missing street': '#ca0020'}

my_plot = sns.barplot(data = ds_st_df, x = 'ord_street_type', y = 'n_segments',
                      hue = 'street_status_desc', palette=my_color_palette)

# set the top part of the plot frame - the spine - to black
ax.spines['top'].set_edgecolor(color = 'black')

y_ticks = list(range(0, 22001, 2000))
y_tick_labels_formatted = ['{:,}'.format(ytl) for ytl in y_ticks]   

my_plot.set_yticks(ticks = y_ticks)
my_plot.set_yticklabels(labels = y_tick_labels_formatted, rotation=0)

my_title = "Total segments by road types and road status"

plt.title(label = my_title, fontsize = 16)
plt.xlabel(xlabel = 'Road type')
plt.ylabel(ylabel = "Number of segments")

plt.legend(title='Road status')

output_file_name = '..\\graphics\\barplot_segment_count.png'
output_file_name = os.path.normpath(output_file_name)
print(output_file_name)
my_plot.get_figure().savefig(fname = output_file_name)
plt.show()